# Web Assessment
- Author: Aidan Marlin
- Date: 20220627
- Version: 1.6
- Description: Jupyter Notebook to automate the execution of web scans

In [ ]:
target = "www.bing.com"
top_target = "bing.com"
ffuf_subdomain_target = "FUZZ.bing.com"
port = 443
### Set to 0 if scanning a subdomain
subdomain_recon = 1

###
### No need to modify from here
###
import socket
import os

os.environ["PATH"] = os.environ["PATH"] + ":/run/current-system/sw/bin"

target_ip = socket.gethostbyname(target) 
ffuf_subdomain_target_https = "https://" + ffuf_subdomain_target
ffuf_subdomain_target_http = "http://" + ffuf_subdomain_target
ffuf_directory_brute = target + ":" + str(port) + "/FUZZ"
http_target = "http://" + target
https_target = "https://" + target

!mkdir -p tools/amass
!mkdir -p tools/arachni
!mkdir -p tools/ctfr
!mkdir -p tools/ffuf_subdomain
!mkdir -p tools/ffuf_directory
!mkdir -p tools/finalrecon
!mkdir -p tools/knockpy
!mkdir -p tools/nettacker
!mkdir -p tools/nikto
!mkdir -p tools/photon
!mkdir -p tools/puppeteer
!mkdir -p tools/subfinder
!mkdir -p tools/sn1per
!mkdir -p tools/sudomy
!mkdir -p tools/testssl
!mkdir -p tools/theharvester

def read_file(location):
    try:
        with open(location) as f:
            text = f.read()
            return text
    except:
        return False


# Target

In [ ]:
print(target)
print(target_ip)


## Whois

In [ ]:
!dig {target}


# Pentester Details

## IP Address

In [ ]:
!curl https://ifconfig.me/


## Start Time

In [ ]:
!date


## Interface Info

In [ ]:
!ip a


# Scans

## HTTP Headers

In [ ]:
%%bash -s "$https_target"
time curl -s --connect-timeout 10 -I $1


## shcheck

In [ ]:
!time docker run --rm shcheck {https_target}


## DrHeader

In [ ]:
!time docker run --entrypoint drheader --rm drheader scan single {https_target}


## ctfr

In [ ]:
!time docker run --rm -v "`pwd`/tools/ctfr/:/mnt" unapibageek/ctfr -d $top_target -o /mnt/results.txt


## Corsy

In [ ]:
!time python "/home/user/git/pentest-tools/Corsy/corsy.py" -u {https_target}:{port}


## waybackpy

In [ ]:
!time docker run --rm secsi/waybackpy --url {target} --oldest


## getjs

In [ ]:
!time docker run --rm secsi/getjs --url {https_target}


## httpx

In [ ]:
!time echo {target} | httpx -silent -status-code -title -tech-detect -ip -asn


## dnstwist

In [ ]:
!time docker run --rm elceef/dnstwist {target}


## Gospider

In [ ]:
!time docker run --rm secsi/gospider -s {https_target}


## CloudFail

In [ ]:
!time docker run --rm secsi/cloudfail -u -t {target}


## CMSeek

In [ ]:
!time docker run --rm secsi/cmseek --batch -u {https_target}


## WhatWeb

In [ ]:
%%bash -s "$https_target" "$port"
time docker run --rm guidelacour/whatweb ./whatweb $1:$2 | sed -r "s/\x1B\[([0-9]{1,3}(;[0-9]{1,2})?)?[mGK]//g"


## hakrawler

In [ ]:
!time echo {https_target} | hakrawler | sort | uniq


## Subfinder

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"
if [[ "$1" == "1" ]]; then
    # Scan
    time subfinder -d $2 -silent -active -oI -oJ -o tools/subfinder/results.txt 2>&1 >/dev/null
    # Output
    cat tools/subfinder/results.txt | jq -r '"Domain:  \(.host)\nIP addr: \(.ip)\n"'
fi


## Backup File Artifacts Checker

In [ ]:
!time docker run --rm secsi/bfac --no-text --url {https_target}


## Screenshot

In [ ]:
import shutil
from pathlib import Path
home = str(Path.home())

os.system("docker run -u root --shm-size 1G --rm -v \"" + os.getcwd() + "/tools/puppeteer:/screenshots\" alekzonder/puppeteer:latest screenshot " + https_target + ":" + str(port) + " 1024x768")
if not os.path.isfile("tools/puppeteer/screenshot_1024_768.png"):
    shutil.copyfile(home + "/git/nixos/resources/404.png", "tools/puppeteer/screenshot_1024_768.png")


![Target screenshot](tools/puppeteer/screenshot_1024_768.png)


## Email Enumeration

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"
if [[ "$1" == "1" ]]; then
    time docker run --rm infoga --domain $2 --source all --breach -v 2
else
    echo "Email enumeration disabled."
fi


## amass

In [ ]:
%%bash -s "$subdomain_recon" "$top_target" "$port"
if [[ "$1" == "1" ]]; then
    time amass enum -silent -nocolor -active -json tools/amass/results.txt -d $2 -p $3
    cat tools/amass/results.txt | jq -r '"##### \(.name)\nIP addr: \(.addresses|.[].ip)\nCIDR: \(.addresses|.[].cidr)\nASN: \(.addresses|.[].asn)\nDesc: \(.addresses|.[].desc)\n"'
else
    echo "DNS enumeration disabled"
fi


## theHarvester

In [ ]:
!time docker run --rm -v "`pwd`/tools/theharvester:/mnt" simonthomas/theharvester:latest theharvester -d $top_target -b all -f /mnt/results.html


## ffuf (subdomains)

In [ ]:
%%bash -s "$subdomain_recon" "$ffuf_subdomain_target"

if [[ "$1" == "1" ]]; then
    echo https://$2
    # Scan
    time ffuf -s -of all -w /home/user/git/wordlists/SecLists/Discovery/DNS/subdomains-top1million-5000.txt -u https://$2 -od tools/ffuf_subdomain
    echo "`find tools/ffuf_subdomain -type f | wc -l` responses worth investigating.."
else
    echo "Subdomain scans have been disabled."
fi


## Knockpy

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"

if [[ "$1" == "1" ]]; then
    time docker run --rm -v "`pwd`/tools/knockpy:/mnt/" secsi/knockpy -o /mnt $2 2>&1 > /dev/null
    find tools/knockpy -type f -exec cat {} \;
else
    echo "Subdomain scans have been disabled."
fi


## Sudomy

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"

API_SHODAN=`cat /etc/api-shodan`
API_VIRUSTOTAL=`cat /etc/api-virustotal`

if [[ "$1" == "1" ]]; then
    time docker run -v "`pwd`/tools/sudomy:/usr/lib/sudomy/output" -e "SHODAN_API=$API_SHODAN" -e "VIRUSTOTAL=$API_VIRUSTOTAL" -t --rm screetsec/sudomy:v1.1.9-dev -d $2 -dP -eP -rS -cF -pS -tO -gW --httpx --dnsprobe  -aI webanalyze --slack -sS
else
    echo "Subdomain scans have been disabled."
fi


## photon

In [ ]:
!time photon --keys -u {https_target} -l 3 -t 100 --wayback -o tools/photon/{target}


### files.txt

In [ ]:
print(read_file("tools/photon/" + target + "/files.txt"))


### fuzzable.txt

In [ ]:
print(read_file("tools/photon/" + target + "/fuzzable.txt"))


### intel.txt

In [ ]:
print(read_file("tools/photon/" + target + "/intel.txt"))


### scripts.txt

In [ ]:
print(read_file("tools/photon/" + target + "/scripts.txt"))


### external.txt

In [ ]:
!time cat tools/photon/{target}/external.txt | httpx -silent -status-code -location -title -tech-detect -cl -ct -fc 401,403,404


### internal.txt

In [ ]:
!time cat tools/photon/{target}/internal.txt | httpx -silent -status-code -location -title -tech-detect -cl -ct -fc 401,403,404


## is-website-vulnerable

In [ ]:
!time docker run --rm lirantal/is-website-vulnerable:latest {https_target}


## dirsearch

In [ ]:
!time docker run --rm secsi/dirsearch -q -x 301 -u {https_target}


## dvcs-ripper

In [ ]:
!time docker run --rm secsi/dvcs-ripper ./rip-svn.pl -u {https_target}


In [ ]:
!time docker run --rm secsi/dvcs-ripper ./rip-cvs.pl -u {https_target}


In [ ]:
!time docker run --rm secsi/dvcs-ripper ./rip-bzr.pl -u {https_target}


In [ ]:
!time docker run --rm secsi/dvcs-ripper ./rip-hg.pl -u {https_target}


In [ ]:
!time docker run --rm secsi/dvcs-ripper ./rip-git.pl -u {https_target}


## Dirhunt

In [ ]:
!time docker run --rm secsi/dirhunt -x not_found --progress-disabled {target} | grep -Ev "No interesting files detected"


## dnscan

In [ ]:
%%bash -s "$subdomain_recon" "$target"

if [[ "$1" == "1" ]]; then
    time docker run --rm secsi/dnscan -d $2 -w subdomains-10000.txt -t 10 -R 1.0.0.1
else
    echo "Subdomain scans have been disabled."
fi


## fierce

In [ ]:
!time docker run --rm secsi/fierce --domain {target}


## Nuclei

In [ ]:
!time docker run --rm -v "`pwd`:/mnt" projectdiscovery/nuclei -u {https_target}:{port}


## TLS Scan

In [ ]:
!time testssl.sh -oA tools/testssl {target}:{port}


## Nettacker

In [ ]:
!time docker run --rm -v "`pwd`/tools/nettacker:/usr/src/owaspnettacker/.data/results/" nettacker python3 ./nettacker.py -m all -i {https_target}


In [ ]:
!echo "Full Nettacker results in file://`pwd`/tools/nettacker"


## Raccoon

In [ ]:
!time docker run --rm evyatarmeged/raccoon:latest {https_target} --ignored-response-codes "301,302,400,401,403,404,503,504"


## ffuf (directory brute)


In [ ]:
!echo https://{ffuf_directory_brute}
!time ffuf -s -ac -sf -of all -w /home/user/git/wordlists/SecLists/Discovery/Web-Content/raft-large-directories.txt -u https://{ffuf_directory_brute} -od tools/ffuf_directory
!echo "`find tools/ffuf_directory -type f | wc -l` responses worth investigating.."
!find tools/ffuf_directory -type f -exec grep -hE 'GET |^HTTP/' {} \; -exec echo \;


## dnsenum

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"

if [[ "$1" == "1" ]]; then
    time docker run --rm guidelacour/dnsenum ./dnsenum.pl $2 --file dns.txt
else
    echo "Subdomain scans have been disabled."
fi


In [ ]:
!find "`pwd`/tools/sudomy" -type f ! -size 0 -exec echo -e "\n###\n### {}\n###" \; -exec cat {} \;


## FinalRecon

In [ ]:
!time docker run --rm -v "`pwd`/tools/finalrecon:/root/.local/share/finalrecon/dumps/" thewhiteh4t/finalrecon --full {https_target}


## Arachni

In [ ]:
!time docker run --rm --net host -v "`pwd`/tools/arachni:/mnt" arachni/arachni:latest /usr/local/arachni/bin/arachni --timeout "0:3:0" --output-only-positives --report-save-path /mnt {https_target}


In [ ]:
!time docker run --rm --net host -v "`pwd`/tools/arachni:/mnt" arachni/arachni:latest find /mnt -type f -exec /usr/local/arachni/bin/arachni_reporter --reporter html:outfile=/mnt/arachni.zip {} \;


In [ ]:
!sudo chown user:users tools/arachni/arachni.zip
!unzip -o tools/arachni/arachni.zip -d tools/arachni/html_report


In [ ]:
!echo Results in file://`pwd`/tools/arachni/html_report/index.html


## Nikto

In [ ]:
!time docker run --rm --net=host -v "`pwd`/tools/nikto:/mnt/" sullo/nikto -ask No -nointeractive -Format json -output /mnt/nikto -h {https_target}


# Manual

## Nessus

Check .zsh_history


## reNgine

In [ ]:
!echo d-rengine


https://localhost:8020/

## Spiderfoot

http://localhost:8060/

## OpenVAS

https://localhost:8030/login/login.html (admin / admin)

## sn1per

Not run automatically because of large output and slow scan time

In [ ]:
!echo docker run -v \"`pwd`/tools/sn1per:/usr/share/sniper/loot/workspace/\" --rm -it xer0dayz/sn1per sniper -t {target}


In [ ]:
!echo find \"$PWD/tools/sn1per/"*"/output\" -type f -exec cat {} \\\;


# Notify

In [ ]:
!/home/user/git/nixos/scripts/telegram_notify.sh -a -m "PT Web Assessment complete for {target}."


# End Time

In [ ]:
!date
